### curriculum - 【AIコース】強化学習　入門編5 Policy Gradient method

In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import gym
import matplotlib.pyplot as plt

env = gym.make("CartPole-v0")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [ ]:
env.reset()
for _ in range(1000):
    env.render()
    env.step(env.action_space.sample())

In [2]:
gamma = 0.99

def discount_rewards(r):
    discounted_r = np.zeros_like(r)
    running_add = 0
    
    for t in reversed(range(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    
    return discounted_r

In [3]:
class agent():
    def __init__(self, lr, s_size, a_size, h_size):
        #C 1. エージェントが環境を入力とし、行動を出力する仕組み
        self.state_in = tf.placeholder(shape = [None, s_size], dtype = tf.float32)
        hidden = slim.fully_connected(self.state_in, 
                                      h_size,
                                      biases_initializer = None,
                                      activation_fn = tf.nn.relu)
        self.output = slim.fully_connected(hidden, 
                                           a_size, 
                                           activation_fn = tf.nn.softmax, 
                                           biases_initializer = None)
        self.chosen_action = tf.argmax(self.output, 1)
        #C 2. REINFORCEのアルゴリズムに従って、policyのgradientをとる仕組み
        self.reward_holder = tf.placeholder(shape = [None], dtype = tf.float32)
        self.action_holder = tf.placeholder(shape = [None], dtype = tf.int32)
        self.indexes = tf.range(0, tf.shape(self.output)[0]) * tf.shape(self.output)[1] + self.action_holder
        self.responsible_outputs = tf.gather(tf.reshape(self.output, [-1]), self.indexes)
        self.loss = -tf.reduce_mean(tf.log(self.responsible_outputs) * self.reward_holder)
        tvars = tf.trainable_variables()
        self.gradient_holders = []
        for idx, var in enumerate(tvars):
            placeholder = tf.placeholder(tf.float32, name = str(idx) + "_holder")
            self.gradient_holders.append(placeholder)
        
        self.gradients = tf.gradients(self.loss, tvars)
        #C 3. 実際にパラメータを更新する仕組み
        optimizer = tf.train.AdamOptimizer(learning_rate = lr)
        self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders, tvars))

In [5]:
tf.reset_default_graph()
myAgent = agent(lr = 1e-2, s_size = env.observation_space.shape[0], a_size = env.action_space.n, h_size = 8)
total_episodes = 5000
max_ep = 999

update_frequency = 5
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    i = 0
    total_reward = []
    total_length = []
    
    grad_holder = sess.run(tf.trainable_variables())
    for ix, grad in enumerate(grad_holder):
        grad_holder[ix] = grad * 0
    
    while i < total_episodes:
        env.render()
        s = env.reset()
        running_reward = 0
        ep_history = []
        for j in range(max_ep):
            a_dist = sess.run(myAgent.output, feed_dict = {myAgent.state_in:[s]})
            a = np.random.choice(a_dist[0], p = a_dist[0])
            a = np.argmax(a_dist == a)
            s1, r, d, _ = env.step(a)
            ep_history.append([s, a, r, s1])
            s = s1
            running_reward += r
            if d == True:
                ep_history = np.array(ep_history)
                ep_history[:, 2] = discount_rewards(ep_history[:, 2])
                feed_dict = {myAgent.reward_holder: ep_history[:, 2],
                             myAgent.action_holder: ep_history[:, 1], 
                             myAgent.state_in: np.vstack(ep_history[:, 0])}
                grads = sess.run(myAgent.gradients, feed_dict = feed_dict)
                for idx, grad in enumerate(grads):
                    grad_holder[idx] += grad
                    
                if i % update_frequency == 0 and i != 0:
                    feed_dict = dictionary = dict(zip(myAgent.gradient_holders, grad_holder))
                    _ = sess.run(myAgent.update_batch, feed_dict = feed_dict)
                    for ix, grad in enumerate(grad_holder):
                        grad_holder[ix] = grad * 0
                total_reward.append(running_reward)
                total_length.append(j)
                break
            
            if i % 100 == 0:
                print(np.mean(total_reward[-100:]))
            i += 1
            

/Users/yasu/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/Users/yasu/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/yasu/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan
22.25
21.11111111111111
22.153846153846153
22.666666666666668
20.88
21.448275862068964
22.29032258064516
22.62857142857143
23.974358974358974
25.170731707317074
25.355555555555554
25.285714285714285
25.442307692307693
25.48148148148148
26.406779661016948
26.161290322580644
26.303030303030305
26.492753623188406
26.527027027027028
26.844155844155843
26.426829268292682
26.546511627906977
27.022727272727273
26.847826086956523
27.157894736842106
27.175257731958762
27.75
28.11
28.17
28.21
28.25
28.55
29.17
29.67
30.19
29.96
30.06
29.98
29.54
29.85
30.12
30.76
31.56
30.65
31.08
31.61
31.92
32.17
32.3
32.82
